<a href="https://colab.research.google.com/github/MenaHagag/Capstone-Project-/blob/master/City_of_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

In [56]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup # beautifulsoup
import requests
print('Libraries imported.')

Libraries imported.


In [0]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text



In [0]:
# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())

In [0]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

 #print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        if (Postcode_var == 'Not assigned\n' or Borough_var == 'Not assigned\n' or Neighbourhood_var == 'Not assigned\n'):
            continue
        if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
            continue
        
        Postcode.append(Postcode_var[:-1])
        Borough.append(Borough_var[:-1])
        Neighbourhood.append(Neighbourhood_var)
   

In [60]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 103


In [61]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(11)

,Postcode,Borough,Neighbourhood
0,M4B,East York,"Parkview Hill, Woodbine Gardens"
1,M9M,North York,"Humberlea, Emery"
2,M1R,Scarborough,"Wexford, Maryvale"
3,M2K,North York,Bayview Village
4,M5N,Central Toronto,Roselawn
5,M6R,West Toronto,"Parkdale, Roncesvalles"
6,M4L,East Toronto,"India Bazaar, The Beaches West"
7,M9N,York,Weston
8,M9P,Etobicoke,Westmount
9,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"


In [62]:
df_toronto.shape

(103, 3)

In [63]:
from google.colab import files
uploaded = files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates (2).csv


In [64]:
df_GC = pd.read_csv("Geospatial_Coordinates.csv")
df_GC.head(11)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [65]:
df_GC.shape

(103, 3)

In [66]:
df_toronto.head(5)

,Postcode,Borough,Neighbourhood
0,M4B,East York,"Parkview Hill, Woodbine Gardens"
1,M9M,North York,"Humberlea, Emery"
2,M1R,Scarborough,"Wexford, Maryvale"
3,M2K,North York,Bayview Village
4,M5N,Central Toronto,Roselawn


In [0]:
df_toronto['Latitude'] = pd.Series([0.00 for x in range(len(df_toronto.index))])

In [68]:
df_toronto['Longitude'] = pd.Series([0.00 for x in range(len(df_toronto.index))])

df_toronto.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4B,East York,"Parkview Hill, Woodbine Gardens",0.0,0.0
1,M9M,North York,"Humberlea, Emery",0.0,0.0
2,M1R,Scarborough,"Wexford, Maryvale",0.0,0.0
3,M2K,North York,Bayview Village,0.0,0.0
4,M5N,Central Toronto,Roselawn,0.0,0.0


In [69]:
for i in range(len(df_toronto.index)):
  ps = df_toronto["Postcode"][i]
  for g in range(len(df_GC.index)):
    psg = df_GC["Postal Code"][g]
    if ps == psg : 
      df_toronto['Latitude'][i] = df_GC['Latitude'][g]
      df_toronto['Longitude'][i] = df_GC['Longitude'][g]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [70]:
df_toronto.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
1,M9M,North York,"Humberlea, Emery",43.724766,-79.532242
2,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
3,M2K,North York,Bayview Village,43.786947,-79.385975
4,M5N,Central Toronto,Roselawn,43.711695,-79.416936
5,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
6,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
7,M9N,York,Weston,43.706876,-79.518188
8,M9P,Etobicoke,Westmount,43.696319,-79.532242
9,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259


In [71]:
df_borough = df_toronto[df_toronto['Borough'].str.find('Toronto')>-1]
df_borough.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
4,M5N,Central Toronto,Roselawn,43.711695,-79.416936
5,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
6,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
14,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
15,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [72]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = df_toronto.drop('Neighbourhood',1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('Borough',1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('Postcode',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 0, 0, 3, 1, 2, 2, 0], dtype=int32)

In [73]:
# add clustering labels
toronto_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)


toronto_grouped_clustering.head() 

,Cluster Labels,Latitude,Longitude
0,1,43.706397,-79.309937
1,2,43.724766,-79.532242
2,1,43.750072,-79.295849
3,0,43.786947,-79.385975
4,0,43.711695,-79.416936


In [74]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [0]:
toronto_grouped_clustering['Neighborhood'] = df_toronto['Neighbourhood']

In [77]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_grouped_clustering['Latitude'], toronto_grouped_clustering['Longitude'],toronto_grouped_clustering['Neighborhood'],  
toronto_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto